# Google form analysis

Analysis of results extracted from Google forms in csv format.

## Preparation

In [1]:
%run "../Utilities/Preparation.ipynb"
%run "../Static data/Google form correct answers.ipynb"

C:\Users\Mikael\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2902: DtypeWarning: Columns (24,32,34) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Constants

In [2]:
#formname = '2017-09-12-gform-fr.csv'
formname = '2017-09-12-gform-en.csv'
gform = pd.read_csv("../../data/Google forms/" + formname)

In [3]:
#localplayerguidkey = 'Ne pas modifier - identifiant anonyme prérempli'
localplayerguidkey = 'Do not edit -  pre-filled anonymous ID'
localplayerguidindex = 34

# Functions
<a id=functions />

In [4]:
#user id in format localplayerguid = '"8d352896-a3f1-471c-8439-0f426df901c1"'

def getAllResponders():
    userIds = gform[localplayerguidkey].values
    result = []
    for id in userIds:
        result.append('"' + id + '"')
    return result

def hasAnswered( userId ):
    return userId.replace('"','') in gform[localplayerguidkey].values

def getAnswers( userId):
    answers = gform[gform[localplayerguidkey]==userId.replace('"','')]
    _columnAnswers = answers.T
    _columnAnswers.columns = ['answers']
    return _columnAnswers

def getCorrections( userId):
    _columnAnswers = getAnswers( userId )
    
    _columnAnswers['correct'] = _columnAnswers['answers']

    for index in range(0, 10):
        _columnAnswers['correct'][index] = 'NaN'
    for index in range(10, len(_columnAnswers.values)-1):
        _columnAnswers['correct'][index] = str(_columnAnswers['answers'][index]).startswith(str(correctAnswers['correctAnswer'][index]))
    _columnAnswers['correct'][len(_columnAnswers.values)-1] = 'NaN'
    return _columnAnswers

#function that returns the score from user id
def getScore( userId ):
    if hasAnswered( userId ):
        _columnAnswers = getCorrections( userId)
        score = (_columnAnswers['correct']).value_counts()[True]
        return score
    else:
        return np.NaN

#function that returns the list of checkpoints from user id
def getValidatedCheckpoints( userId ):
    _columnAnswers = getCorrections( userId)
    questionnaireValidatedCheckpointsPerQuestion = pd.Series(np.nan, index=range(len(checkpointQuestionMatching)))
    
    for index in range(0, len(questionnaireValidatedCheckpointsPerQuestion)):
        if _columnAnswers['correct'][index]==True:
            questionnaireValidatedCheckpointsPerQuestion[index] = checkpointQuestionMatching['checkpoint'][index]
        else:
            questionnaireValidatedCheckpointsPerQuestion[index] = ''
    
    questionnaireValidatedCheckpoints = questionnaireValidatedCheckpointsPerQuestion.unique()
    questionnaireValidatedCheckpoints = questionnaireValidatedCheckpoints[questionnaireValidatedCheckpoints!='']
    questionnaireValidatedCheckpoints = pd.Series(questionnaireValidatedCheckpoints)
    questionnaireValidatedCheckpoints = questionnaireValidatedCheckpoints.sort_values()
    questionnaireValidatedCheckpoints.index = range(0, len(questionnaireValidatedCheckpoints))
    return questionnaireValidatedCheckpoints

def getNonValidated( checkpoints ):
    return pd.Series(np.setdiff1d(checkpointQuestionMatching['checkpoint'].values, checkpoints.values))

def getNonValidatedCheckpoints( userId ):
    validated = getValidatedCheckpoints( userId )
    return getNonValidated(validated)